In [ ]:
import pandas as pd

########################################## LENDO A BASE OPERACIONAL PRONTA PARA TRABALHAR
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO"
base_operacional_hugo = pd.read_csv(caminho+"/BASE_OPERACIONAL_TELECOB_HUGO.txt", sep=';', dtype=str,decimal=',')

base_operacional_hugo['ATRASO'] = base_operacional_hugo['ATRASO'].astype('int16')
base_operacional_hugo['SALDO_ABERTO'] = base_operacional_hugo['SALDO_ABERTO'].str.replace('.', '').str.replace(',', '.').astype(float)
base_operacional_hugo['BASE'] = base_operacional_hugo['BASE'].apply(lambda x: x.replace('Claro MÃ³vel','Claro Móvel'))
base_operacional_hugo = base_operacional_hugo.assign(FILA="")



###################### CRIANDO OS LAYOUTS PARA OS MAILINGS
preditivo_higienizado_claro = ["CPF","NOME_x","DEVEDOR","BASE","FILA","ATRASO","SALDO_ABERTO","TELEFONE_HIGIENIZADO"]
preditivo_validados_claro = ["CPF","NOME_x","DEVEDOR","BASE","FILA","ATRASO","SALDO_ABERTO",'TELEFONE',"TELEFONE_HIGIENIZADO"]
preditivo_telefones_intersic = ["CPF","NOME_x","DEVEDOR","BASE","FILA","ATRASO","SALDO_ABERTO",'TELEFONE']

In [ ]:

## identificando clientes com mais de um contrato..
base_mais_contratos = base_operacional_hugo.drop_duplicates(["CPF",'BASE'])
base_mais_contratos['duplicado'] = base_mais_contratos.groupby('CPF')['CPF'].transform('count') > 1

## FILTRANDO TUDO QUE FOR VERDADEIRO NA DUPLICIDADE
base_mais_contratos = base_mais_contratos.sort_values(['CPF']).query("duplicado == True")

## INIBINDO CLIENTES GEVENUE
base_mais_contratos = base_mais_contratos.query("BASE != 'GGEVENUE NET' and BASE != 'GEVENUE CLARO MOVEL'")

## identificando clientes com mais de um contrato..
base_mais_contratos = base_mais_contratos.drop_duplicates(["CPF",'BASE'])
base_mais_contratos['duplicado'] = base_mais_contratos.groupby('CPF')['CPF'].transform('count') > 1
base_mais_contratos = base_mais_contratos.query("duplicado == True").sort_values(['CPF'])


## filtrando apenas a coluna cpf e duplicado para marcar o que for TRUE na base total e poder fazer o mailing apartado.
base_mais_contratos = base_mais_contratos[['CPF','duplicado']].drop_duplicates(['CPF']).reset_index(drop=True)


base_operacional_hugo = pd.merge(base_operacional_hugo, base_mais_contratos, right_on=['CPF'], left_on=['CPF'], how="left")
base_mais_contratos = base_operacional_hugo.query("duplicado == True")
base_mais_contratos = base_mais_contratos.query("BASE != 'GEVENUE NET' and BASE != 'GEVENUE CLARO MOVEL'")

base_mais_contratos = base_mais_contratos.sort_values(['CPF'])


### Mantendo na base operacional hugo 
base_operacional_hugo = base_operacional_hugo.query("duplicado.isnull()").reset_index(drop=True)


In [ ]:
### Mailings tel's higienizados clientes com mais contratos
base_mais_contratos_higienizados = base_mais_contratos.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
base_mais_contratos_higienizados = base_mais_contratos_higienizados.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

### Tirando os telefones nulos
base_mais_contratos_higienizados = base_mais_contratos_higienizados.query("TELEFONE_HIGIENIZADO.notnull()")

base_mais_contratos_higienizados_0a1785 = base_mais_contratos_higienizados.query("ATRASO <= 1785")


### Mailings tel's CONTRATANTE clientes com mais contratos
base_mais_contratos_contratante = base_mais_contratos.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
base_mais_contratos_contratante = base_mais_contratos_contratante.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])

base_mais_contratos_contratante_0a1785 = base_mais_contratos_contratante.query("ATRASO <= 1785")

#### MAILINGS PJ, GEVENUE E INTERSIC

In [ ]:
# MAILINGS PJ CLARO / NET E GEVENUE BB
base_operacional_pj  = base_operacional_hugo[base_operacional_hugo['CPF'].str.len()==14]

In [ ]:
### SEPARANDO MAILING PJ NET GEVENUE
base_gevenue_net_pj = base_operacional_pj.query("BASE == 'GEVENUE NET' and  ATRASO <= 1785 and   SALDO_ABERTO >= 20  ")

gevenue_net_contratante_pj = base_gevenue_net_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
gevenue_net_higi_pj = base_gevenue_net_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]


### SEPARANDO MAILING PJ GEVENUE NET contratante

gevenue_net_contratante_pj = gevenue_net_contratante_pj.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
gevenue_net_contratante_pj = gevenue_net_contratante_pj.query("TELEFONE.notnull()")
gevenue_net_contratante_pj = gevenue_net_contratante_pj.sample(frac = 1)

gevenue_net_contratante_pj_0a720 = gevenue_net_contratante_pj.query("BASE == 'GEVENUE NET' and  ATRASO <= 720")
gevenue_net_contratante_pj_721a1785 = gevenue_net_contratante_pj.query("BASE == 'GEVENUE NET' and  ATRASO >= 721")


### higienziado pj 
gevenue_net_higi_pj = gevenue_net_higi_pj.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
gevenue_net_higi_pj = gevenue_net_higi_pj.query("TELEFONE_HIGIENIZADO.notnull()")
gevenue_net_higi_pj = gevenue_net_higi_pj.sample(frac = 1)

gevenue_net_higi_pj_0a720 = gevenue_net_higi_pj.query("BASE == 'GEVENUE NET' and  ATRASO <= 720")
gevenue_net_higi_pj_721a1785 = gevenue_net_higi_pj.query("BASE == 'GEVENUE NET' and  ATRASO >= 721")


In [ ]:
### SEPARANDO MAILING PJ GEVENUE CLARO MOVEL
base_gevenue_claro_movel_pj = base_operacional_pj.query("BASE == 'GEVENUE CLARO MOVEL' and  ATRASO <= 1785 and   SALDO_ABERTO >= 20  ")

#### mailing contratante
gevenue_claro_movel_contratante_pj_0a1785 = base_gevenue_claro_movel_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
gevenue_claro_movel_contratante_pj_0a1785 = gevenue_claro_movel_contratante_pj_0a1785.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
gevenue_claro_movel_contratante_pj_0a1785 = gevenue_claro_movel_contratante_pj_0a1785.sample(frac = 1)


#### mailing higienizado
gevenue_claro_movel_higi_pj_0a1785 = base_gevenue_claro_movel_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
gevenue_claro_movel_higi_pj_0a1785 = gevenue_claro_movel_higi_pj_0a1785.query("TELEFONE_HIGIENIZADO.notnull()")

gevenue_claro_movel_higi_pj_0a1785 = gevenue_claro_movel_higi_pj_0a1785.sample(frac = 1)

In [ ]:
### SEPARANDO MAILING PJ INTERSIC 

bases_intersic = ['Claro TV', 'Claro Móvel', 'Claro Fixo']

base_intersic_claro_pj = base_operacional_pj.query(f"BASE in {bases_intersic} and  ATRASO <= 1785 and   SALDO_ABERTO >= 20  ")
base_intersic_claro_pj = base_intersic_claro_pj.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])

### Mailings claro pj intersic CONTRATANTE

claro_intersic_contratante_pj_0a1785 = base_intersic_claro_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
claro_intersic_contratante_pj_0a1785 = claro_intersic_contratante_pj_0a1785.sample(frac = 1)

### Mailings claro pj intersic HIGIENIZADO

claro_intersic_higi_pj_0a1785 = base_intersic_claro_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
claro_intersic_higi_pj_0a1785 = claro_intersic_higi_pj_0a1785.query("TELEFONE_HIGIENIZADO.notnull()")
claro_intersic_higi_pj_0a1785 = claro_intersic_higi_pj_0a1785.sample(frac = 1)

#### MAILINGS CLARO INTERSIC PESSOA FISICA

In [ ]:
##### SEPARANDO A BASE CLARO MOVEL

base_intersic_movel = base_operacional_hugo.query("BASE == 'Claro Móvel' and  ATRASO <= 1785 and   SALDO_ABERTO >= 20  ")


claro_movel_intersic_contratante = base_intersic_movel.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
claro_movel_intersic_contratante = claro_movel_intersic_contratante.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])

####### MAILINGS CONTRATANTE
claro_movel_intersic_contratante_0a200 = claro_movel_intersic_contratante.query("BASE == 'Claro Móvel' and  ATRASO <= 200")
claro_movel_intersic_contratante_0a200 = claro_movel_intersic_contratante_0a200.sample(frac = 1)

claro_movel_intersic_contratante_201a360 = claro_movel_intersic_contratante.query("BASE == 'Claro Móvel' and  ATRASO >= 201 and ATRASO <= 360")
claro_movel_intersic_contratante_201a360 = claro_movel_intersic_contratante_201a360.sample(frac = 1)

claro_movel_intersic_contratante_361a720 = claro_movel_intersic_contratante.query("BASE == 'Claro Móvel' and  ATRASO >= 361 and ATRASO <= 720")
claro_movel_intersic_contratante_361a720 = claro_movel_intersic_contratante_361a720.sample(frac = 1)

claro_movel_intersic_contratante_721a1080 = claro_movel_intersic_contratante.query("BASE == 'Claro Móvel' and  ATRASO >= 721 and ATRASO <= 1080")
claro_movel_intersic_contratante_721a1080 = claro_movel_intersic_contratante_721a1080.sample(frac = 1)

claro_movel_intersic_contratante_1081a1785 = claro_movel_intersic_contratante.query("BASE == 'Claro Móvel' and  ATRASO >= 1081 and ATRASO <= 1785")
claro_movel_intersic_contratante_1081a1785 = claro_movel_intersic_contratante_1081a1785.sample(frac = 1)



##### MAILINGS HIGIENIZADOS
claro_movel_intersic_higi = base_intersic_movel.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
claro_movel_intersic_higi = claro_movel_intersic_higi.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
claro_movel_intersic_higi = claro_movel_intersic_higi.query("TELEFONE_HIGIENIZADO.notnull()")

claro_movel_intersic_higi_0a200 = claro_movel_intersic_higi.query("BASE == 'Claro Móvel' and  ATRASO <= 200")
claro_movel_intersic_higi_0a200 = claro_movel_intersic_higi_0a200.sample(frac = 1)

claro_movel_intersic_higi_201a360 = claro_movel_intersic_higi.query("BASE == 'Claro Móvel' and  ATRASO >= 201 and ATRASO <= 360")
claro_movel_intersic_higi_201a360 = claro_movel_intersic_higi_201a360.sample(frac = 1)

claro_movel_intersic_higi_361a720 = claro_movel_intersic_higi.query("BASE == 'Claro Móvel' and  ATRASO >= 361 and ATRASO <= 720")
claro_movel_intersic_higi_361a720 = claro_movel_intersic_higi_361a720.sample(frac = 1)

claro_movel_intersic_higi_721a1080 = claro_movel_intersic_higi.query("BASE == 'Claro Móvel' and  ATRASO >= 721 and ATRASO <= 1080")
claro_movel_intersic_higi_721a1080 = claro_movel_intersic_higi_721a1080.sample(frac = 1)

claro_movel_intersic_higi_1081a1785 = claro_movel_intersic_higi.query("BASE == 'Claro Móvel' and  ATRASO >= 1081 and ATRASO <= 1785")
claro_movel_intersic_higi_1081a1785 = claro_movel_intersic_higi_1081a1785.sample(frac = 1)


In [ ]:
##### SEPARANDO A BASE CLARO FIXO

base_intersic_fixo = base_operacional_hugo.query("BASE == 'Claro Fixo' and  ATRASO <= 1785 and   SALDO_ABERTO >= 20  ")


claro_fixo_intersic_contratante_0a1785 = base_intersic_fixo.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
claro_fixo_intersic_contratante_0a1785 = claro_fixo_intersic_contratante_0a1785.sample(frac = 1)

claro_fixo_intersic_higi_0a1785 = base_intersic_fixo.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
claro_fixo_intersic_higi_0a1785 = claro_fixo_intersic_higi_0a1785.sample(frac = 1)

claro_fixo_intersic_higi_0a1785 = claro_fixo_intersic_higi_0a1785.query("TELEFONE_HIGIENIZADO.notnull()")

In [ ]:
##### SEPARANDO A BASE CLARO TV

base_intersic_tv = base_operacional_hugo.query("BASE == 'Claro TV' and  ATRASO <= 1785 and   SALDO_ABERTO >= 20")


claro_tv_intersic_contratante_0a1785 = base_intersic_tv.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
claro_tv_intersic_contratante_0a1785 = claro_tv_intersic_contratante_0a1785.sample(frac = 1)

claro_tv_intersic_higi_0a1785 = base_intersic_tv.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
claro_tv_intersic_higi_0a1785 = claro_tv_intersic_higi_0a1785.query("TELEFONE_HIGIENIZADO.notnull()")
claro_tv_intersic_higi_0a1785 = claro_tv_intersic_higi_0a1785.sample(frac = 1)


#### MAILINGS GEVENUE CLARO MOVEL PF

In [ ]:
### SEPARANDO MAILING PJ GEVENUE CLARO MOVEL
base_gevenue_movel = base_operacional_hugo.query("BASE == 'GEVENUE CLARO MOVEL' and  ATRASO <= 1785 and  SALDO_ABERTO >= 20")

gevenue_movel_contratante_0a1785 = base_gevenue_movel.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
gevenue_movel_contratante_0a1785 = gevenue_movel_contratante_0a1785.sample(frac = 1)


gevenue_movel_higi_0a1785 = base_gevenue_movel.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
gevenue_movel_higi_0a1785 = gevenue_movel_higi_0a1785.query("TELEFONE_HIGIENIZADO.notnull()")
gevenue_movel_higi_0a1785 = gevenue_movel_higi_0a1785.sample(frac = 1)


#### MAILINGS GEVENUE NET PF

In [ ]:
base_gevenue_net = base_operacional_hugo.query("BASE == 'GEVENUE NET' and  ATRASO <= 1785 and  SALDO_ABERTO >= 20")


base_gevenue_net_contratante = base_gevenue_net.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

base_gevenue_net_contratante = base_gevenue_net_contratante.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])

####### MAILINGS CONTRATANTE

gevenue_net_contratante_0a360 = base_gevenue_net_contratante.query("BASE == 'GEVENUE NET' and  ATRASO >= 0 and ATRASO <= 360")
gevenue_net_contratante_0a360 = gevenue_net_contratante_0a360.sample(frac = 1)

gevenue_net_contratante_361a720 = base_gevenue_net_contratante.query("BASE == 'GEVENUE NET' and  ATRASO >= 361 and ATRASO <= 720")
gevenue_net_contratante_361a720 = gevenue_net_contratante_361a720.sample(frac = 1)

gevenue_net_contratante_721a1080 = base_gevenue_net_contratante.query("BASE == 'GEVENUE NET' and  ATRASO >= 721 and ATRASO <= 1080")
gevenue_net_contratante_721a1080 = gevenue_net_contratante_721a1080.sample(frac = 1)

gevenue_net_contratante_1081a1785 = base_gevenue_net_contratante.query("BASE == 'GEVENUE NET' and  ATRASO >= 1081 and ATRASO <= 1785")
gevenue_net_contratante_1081a1785 = gevenue_net_contratante_1081a1785.sample(frac = 1)


##### MAILINGS HIGIENIZADOS
base_gevenue_net_higi = base_gevenue_net.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
base_gevenue_net_higi = base_gevenue_net_higi.query("TELEFONE_HIGIENIZADO.notnull()")

gevenue_net_higi_0a360 = base_gevenue_net_higi.query("BASE == 'GEVENUE NET' and  ATRASO >= 0 and ATRASO <= 360")
gevenue_net_higi_0a360 = gevenue_net_higi_0a360.sample(frac = 1)

gevenue_net_higi_361a720 = base_gevenue_net_higi.query("BASE == 'GEVENUE NET' and  ATRASO >= 361 and ATRASO <= 720")
gevenue_net_higi_361a720 = gevenue_net_higi_361a720.sample(frac = 1)

gevenue_net_higi_721a1080 = base_gevenue_net_higi.query("BASE == 'GEVENUE NET' and  ATRASO >= 721 and ATRASO <= 1080")
gevenue_net_higi_721a1080 = gevenue_net_higi_721a1080.sample(frac = 1)

gevenue_net_higi_1081a1785 = base_gevenue_net_higi.query("BASE == 'GEVENUE NET' and  ATRASO >= 1081 and ATRASO <= 1785")
gevenue_net_higi_1081a1785 = gevenue_net_higi_1081a1785.sample(frac = 1)

In [ ]:
#########################################################################################################################################################
#################################################################### BAIXANDO OS MAILINGS  ##################################################################

# ##################### CLARO MOVEL INTERSIC

claro_movel_intersic_contratante_0a200.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/claro_movel_intersic_contratante_0a200_2902.csv", index=False,sep=';')
claro_movel_intersic_contratante_201a360.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/claro_movel_intersic_contratante_201a360_2902.csv", index=False,sep=';')
claro_movel_intersic_contratante_361a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/claro_movel_intersic_contratante_361a720_2902.csv", index=False,sep=';')
claro_movel_intersic_contratante_721a1080.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/claro_movel_intersic_contratante_721a1080_2902.csv", index=False,sep=';')
claro_movel_intersic_contratante_1081a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/claro_movel_intersic_contratante_1081a1785_2902.csv", index=False,sep=';')

claro_movel_intersic_higi_0a200.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/claro_movel_intersic_higi_0a200_2902.csv", index=False,sep=';')
claro_movel_intersic_higi_201a360.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/claro_movel_intersic_higi_201a360_2902.csv", index=False,sep=';')
claro_movel_intersic_higi_361a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/claro_movel_intersic_higi_361a720_2902.csv", index=False,sep=';')
claro_movel_intersic_higi_721a1080.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/claro_movel_intersic_higi_721a1080_2902.csv", index=False,sep=';')
claro_movel_intersic_higi_1081a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/claro_movel_intersic_higi_1081a1785_2902.csv", index=False,sep=';')

### CLARO TV INTERSIC
claro_tv_intersic_contratante_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/claro_tv_intersic_contratante_0a1785_2902.csv", index=False,sep=';')
claro_tv_intersic_higi_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/claro_tv_intersic_higi_0a1785_2902.csv", index=False,sep=';')

###################### CLARO FIXO INTERSIC
claro_fixo_intersic_contratante_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/claro_fixo_intersic_contratante_0a1785_2902.csv", index=False,sep=';')
claro_fixo_intersic_higi_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/claro_fixo_intersic_higi_0a1785_2902.csv", index=False,sep=';')




############## MAILINGS CLIENTES COM MAIS DE UM CONTRATO

base_mais_contratos_contratante_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\MAIS DE UM CONTRATO/base_mais_contratos_contratante_0a1785_2902.csv", index=False,sep=';')
base_mais_contratos_higienizados_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\MAIS DE UM CONTRATO/base_mais_contratos_higienizados_0a1785_2902.csv", index=False,sep=';')


# # # ###################### GEVENUE CLARO MOVEL

gevenue_movel_contratante_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/gevenue_movel_contratante_0a1785_2902.csv", index=False,sep=';')
gevenue_movel_higi_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/gevenue_movel_higi_0a1785_2902.csv", index=False,sep=';')


####### GEVENUE NET

gevenue_net_contratante_0a360.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/gevenue_net_contratante_0a360_2902.csv", index=False,sep=';')
gevenue_net_contratante_361a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/gevenue_net_contratante_361a720_2902.csv", index=False,sep=';')
gevenue_net_contratante_721a1080.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/gevenue_net_contratante_721a1080_2902.csv", index=False,sep=';')
gevenue_net_contratante_1081a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/gevenue_net_contratante_1081a1785_2902.csv", index=False,sep=';')

gevenue_net_higi_0a360.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/ggevenue_net_higi_0a360_2902.csv", index=False,sep=';')
gevenue_net_higi_361a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/gevenue_net_higi_361a720_2902.csv", index=False,sep=';')
gevenue_net_higi_721a1080.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/gevenue_net_higi_721a1080_2902.csv", index=False,sep=';')
gevenue_net_higi_1081a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/ggevenue_net_higi_1081a1785_2902.csv", index=False,sep=';')



In [ ]:
# MAILINGS DE PJ 

## MAILING PJ GEVENUE CLARO MOVEL

gevenue_claro_movel_contratante_pj_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ\GEVENUE CLARO MOVEL PJ/gevenue_claro_movel_contratante_pj_0a1785_2902.csv", index=False,sep=';')
gevenue_claro_movel_higi_pj_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ\GEVENUE CLARO MOVEL PJ/gevenue_claro_movel_higi_pj_0a1785_2902.csv", index=False,sep=';')


## MAILING PJ GEVENUE NET

gevenue_net_contratante_pj_0a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ\GEVENUE NET PJ/gevenue_net_contratante_pj_0a720_2902.csv", index=False,sep=';')
gevenue_net_contratante_pj_721a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ\GEVENUE NET PJ/gevenue_net_contratante_pj_721a1785_2902.csv", index=False,sep=';')

gevenue_net_higi_pj_0a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ\GEVENUE NET PJ/gevenue_net_higi_pj_0a720_2902.csv", index=False,sep=';')
gevenue_net_higi_pj_721a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ\GEVENUE NET PJ/gevenue_net_higi_pj_721a1785_2902.csv", index=False,sep=';')

## MAILING PJ CLARO INTERSIC

claro_intersic_contratante_pj_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ\INTERSIC CLARO  PJ/claro_intersic_contratante_pj_0a1785_2902.csv", index=False,sep=';')
claro_intersic_higi_pj_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ\INTERSIC CLARO  PJ/claro_intersic_higi_pj_0a1785_2902.csv", index=False,sep=';')


### RETORNO DOS HIGIENIZADOS QUE A OPERAÇÃO PEDE:

In [ ]:
###### BAIXANDO OS EMAILS DA GEVENUE HIGIENIZADOS (DAIANA PEDE PRA AÇÃO MASSIVA)
bases_gevenue_list = ['GEVENUE NET','GEVENUE CLARO MOVEL']

## SEPARANDO OS EMAILS HIGIENIZADOS 
emails_gevenue_higienizados = base_operacional_hugo.query(f"BASE in {bases_gevenue_list} and  EMAIL_HIGIENIZADO.notnull()").loc[:,['CPF','NOME_x','EMAIL_HIGIENIZADO','ATRASO','BASE','CONTRATO']]\
.drop_duplicates(['CPF','NOME_x','EMAIL_HIGIENIZADO'])

##### SEPARANDO OS EMAILS DO CONTRATANTE
emails_gevenue_contratante = base_operacional_hugo.query(f"BASE in {bases_gevenue_list} and  EMAIL.notnull()").loc[:,['CPF','NOME_x','EMAIL','ATRASO','BASE','CONTRATO']]\
.drop_duplicates(['CPF','NOME_x','EMAIL'])


###RENOMEANDO PARA CONCATENAR OS DOIS HIGIENIZADOS E CONTRATANTE
emails_gevenue_higienizados.rename(columns={'EMAIL_HIGIENIZADO': 'EMAIL'}, inplace=True)

# CONCATENANDO, REMOVENDO DUPLICADO E RESETANDO O INDEX
emails_gevenue_devolver = pd.concat([emails_gevenue_higienizados,emails_gevenue_contratante])
emails_gevenue_devolver = emails_gevenue_devolver.drop_duplicates(['CPF','NOME_x','EMAIL','CONTRATO']).reset_index(drop=True)



###### FILTRANDO O QUE É = CPF (11 DIGITOS) PORQUE A QUANTIDDE ULTRAPASSA 1 MILHÃO DE LINHAS
### ORDENANDO POR ATRASO DO MENOR PARA O MAIOR E CPF , FILTRANDO SOMENTE AS LINHAS DE EMAIL QUE NÃO ESTÃO VAZIAS.
## DIVIDINDO EM 2 ARQUIVOS PARA ABRIREM EM EXCEL
emails_gevenue_devolver_cpf = emails_gevenue_devolver[emails_gevenue_devolver['CPF'].str.len()==11].reset_index(drop=True)
emails_gevenue_devolver_cpf = emails_gevenue_devolver_cpf.sort_values(['ATRASO','CPF']).query("EMAIL !=' '")
emails_gevenue_cpf_1 = emails_gevenue_devolver_cpf.iloc[0:999999]
emails_gevenue_cpf_2 = emails_gevenue_devolver_cpf.iloc[1000000:2000000]
emails_gevenue_cpf_3 = emails_gevenue_devolver_cpf.iloc[2000001:3000000]

###### FILTRANDO O QUE É = CNPJ (14 DIGITOS).
#### ORDENANDO POR ATRASO DO MENOR PARA O MAIOR E CPF , FILTRANDO SOMENTE AS LINHAS DE EMAIL QUE NÃO ESTÃO VAZIAS.
emails_gevenue_devolver_cnpj = emails_gevenue_devolver[emails_gevenue_devolver['CPF'].str.len()==14].reset_index(drop=True)
emails_gevenue_devolver_cnpj = emails_gevenue_devolver_cnpj.sort_values(['ATRASO','CPF']).query("EMAIL !=' '")



##### BAIXANDO O ARQUIVO COM 3 ABAS, DUAS CONTENDO EMAILS DE CLIENTES PESSOA FISICA E UMA COM PESSOA JURIDICA.
writer = pd.ExcelWriter(r'R:\TI\TELEFONIA\Mailings Prontos\2024\Março\01\Jorgean/Emails_Gevenue_0103.xlsx', engine='xlsxwriter')

#Salvando o Relatorio na pasta.
emails_gevenue_cpf_1.to_excel(writer, sheet_name='Emails_Gevenue_Cpf_1', index=False)
emails_gevenue_cpf_2.to_excel(writer, sheet_name='Emails_Gevenue_Cpf_2', index=False)
emails_gevenue_cpf_3.to_excel(writer, sheet_name='Emails_Gevenue_Cpf_3', index=False)
emails_gevenue_devolver_cnpj.to_excel(writer, sheet_name='Emails_Gevenue_Cnpj', index=False)
writer.save()

In [ ]:
bases_intersic = ['Claro TV', 'Claro Móvel', 'Claro Fixo']

#### BAIXANDO OS EMAILS DA CLARO MOVEL, FIXO E TV
emails_claro = base_operacional_hugo.loc[:,['CPF','NOME_x','EMAIL_HIGIENIZADO','ATRASO','BASE','CONTRATO']]\
.query("EMAIL_HIGIENIZADO.notnull()").drop_duplicates(['CPF','EMAIL_HIGIENIZADO']).reset_index(drop=True)
## Filtrando tudo exceto Gevenue
emails_claro = emails_claro.query(f"BASE in {bases_intersic}")

emails_claro_1 = emails_claro[:1000000]
emails_claro_2 = emails_claro[1000001:2000000]


##### BAIXANDO O ARQUIVO COM 3 ABAS, DUAS CONTENDO EMAILS DE CLIENTES PESSOA FISICA E UMA COM PESSOA JURIDICA.
writer = pd.ExcelWriter(r'R:\TI\TELEFONIA\Mailings Prontos\2024\Março\01\Jorgean/Emails_Claro_0103.xlsx', engine='xlsxwriter')

#Salvando o Relatorio na pasta.
emails_claro_1.to_excel(writer, sheet_name='emails_CLAROS_FULL_1', index=False)
emails_claro_2.to_excel(writer, sheet_name='emails_CLAROS_FULL_2', index=False)
writer.save()

####  Analises base carteiras

<!-- ### Analise da base operacional -->

In [ ]:
# Criando coluna Faixa de atraso com suas respectivas faixas
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(-1000, -1),    'Faixa_de_Atraso'] = "Atraso (Menor que 0)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(0, 2000),    'Faixa_de_Atraso'] = "Atraso (0 a 2000)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(2001, 2250),    'Faixa_de_Atraso'] = "Atraso (2001 a 2250)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(2501, 3500),    'Faixa_de_Atraso'] = "Atraso (2501 a 3500)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(3501, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 3501)"


###################################################### TELEFONES INTERSIC POR BASE E FAIXA DE ATRASO #####################################################

# FILTRAMOS PRIMEIRO OS TELEFONES DO CONTRATANTE NÃO NULOS, PARA GARANTIR A CONTAGEM DAS LINHAS COM TELEFONE DE FATO.
# SEPARAMOS APENAS AS COLUNAS PARA A ANALISE DE BASE / TELEFONES INTERSIC POR FAIXA DE ATRASO.
# REMOVEMOS OS DUPLICADOS POR CPF/NOME/TELEFONE, POIS RECEBEMOS MUITOS TELEFONES DO MESMO CPF, E EFETUANDO OS CRUZAMENTOS PARA UNIR TUDO OS CLIENTES FICAM DUPLICADOS, O QUE MUDA O NUMERO REAL.
## POR FIM AGRUPO BASE (CARTEIRA) POR FAIXA DE ATRASO, PRA SABER A QUANTIDADE DE TELEFONE DO CONTRATANTE POR CPF 
analitic_telefones_intersic = base_operacional_hugo.query("TELEFONE.notnull()").loc[:,['BASE','Faixa_de_Atraso','CPF','NOME_x','TELEFONE']]\
    .drop_duplicates(['CPF','NOME_x','TELEFONE'])\
    .groupby(['BASE','Faixa_de_Atraso'])['TELEFONE'].count().reset_index().sort_values(['TELEFONE'])


## pula linha no print
'\n'

###################################################### TELEFONES HIGIENIZADOS POR BASE E FAIXA DE ATRASO #####################################################

# FILTRAMOS PRIMEIRO OS HIGIENIZADOS NÃO NULOS, PORQUE NEM TODOS OS CPFS ENVIADOS PRA HIGIENIZAR A AFINDER TEM TELEFONE
# SEPARAMOS APENAS AS COLUNAS PARA A ANALISE DE BASE / TELEFONE HIGIENIZADO POR FAIXA DE ATRASO.
# REMOVEMOS OS DUPLICADOS POR CPF/NOME/TELEFONE, POIS RECEBEMOS MUITOS TELEFONES DO MESMO CPF, E EFETUANDO OS CRUZAMENTOS PARA UNIR TUDO OS CLIENTES FICAM DUPLICADOS, O QUE MUDA O NUMERO REAL.
## POR FIM AGRUPO BASE (CARTEIRA) POR FAIXA DE ATRASO, PRA SABER A QUANTIDADE DE TELEFONE HIGIENIZADO POR CPF 
analitic_telefones_higienizados = base_operacional_hugo.query("TELEFONE_HIGIENIZADO.notnull()").loc[:,['BASE','Faixa_de_Atraso','CPF','NOME_x','TELEFONE_HIGIENIZADO']]\
    .drop_duplicates(['CPF','NOME_x','TELEFONE_HIGIENIZADO'])\
    .groupby(['BASE','Faixa_de_Atraso'])['TELEFONE_HIGIENIZADO'].count().reset_index().sort_values(['TELEFONE_HIGIENIZADO'])


## pula linha no print
'\n'

# ######################### ANÁLISE DE DÉBITO POR FAIXA DE ATRASO

####### SEPARAMOS AS COLUNAS NECESSARIAS PARA ANALISAR POR FAIXA DE ATRASO O SALDO ABERTO DO CLIENTE
##### REMOVEMOS OS DUPLICADOS POR CPF E ATRASO, PARA QUE SE O CLIENTE TIVER MAIS DE UMA FATURA E CASO ENTRE EM UMA OUTRA FAIXA DE ATRASO SEJA CALCULADO O NUMERO VERDADEIRO
analitic_SaldoPorCpf = base_operacional_hugo.loc[:,['BASE','Faixa_de_Atraso','CPF','ATRASO','SALDO_ABERTO']].drop_duplicates(['CPF','ATRASO'])
### FAÇO UM REPLACE NA VIRGULA PARA PONTO, POIS SÓ ASSIM CONSIGO CONVERTER A COLUNA SALDO_ABERTO PARA FLOAT
# analitic_SaldoPorCpf['SALDO_ABERTO'] = analitic_SaldoPorCpf['SALDO_ABERTO'].str.replace('.', '').str.replace(',', '.').astype(float)

analitic_SaldoPorCpf['SALDO_ABERTO'] = analitic_SaldoPorCpf['SALDO_ABERTO'].astype('Float64')

# Criando coluna Faixa de atraso com suas respectivas faixas
analitic_SaldoPorCpf.loc[analitic_SaldoPorCpf.SALDO_ABERTO.between(-99999, 19.99),    'Faixa_de_Débito'] = "Saldo (Menor que 20,00)"
analitic_SaldoPorCpf.loc[analitic_SaldoPorCpf.SALDO_ABERTO.between(20.0, 9999999),    'Faixa_de_Débito'] = "Saldo (Maior que 20,00)"

analitic_SaldoPorCpf_group = analitic_SaldoPorCpf.groupby(['BASE','Faixa_de_Atraso','Faixa_de_Débito'])['CPF'].count().reset_index()

##### JUNTANDO OS TELEFONES DO CONTRATANTE E OS HIGIENIZADOS 
analitic_full_bases = pd.merge(analitic_telefones_intersic, analitic_telefones_higienizados, right_on=['BASE','Faixa_de_Atraso'], left_on=['BASE','Faixa_de_Atraso'], how="inner")


In [ ]:
analitic_telefones_bases_atraso = pd.pivot_table(analitic_full_bases, values=['TELEFONE','TELEFONE_HIGIENIZADO'], index=['BASE'],
                       columns=['Faixa_de_Atraso']).fillna(0)
analitic_telefones_bases_atraso

In [ ]:
analitic_SaldoPorCpf_group = pd.pivot_table(analitic_SaldoPorCpf_group, values=['CPF'], index=['BASE','Faixa_de_Débito'],
                       columns=['Faixa_de_Atraso']).fillna(0)

In [ ]:
# Carregar o arquivo Excel
writer = pd.ExcelWriter(r'R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\8\Analitic_Telecobs_Hugo_0808.xlsx', engine='xlsxwriter')

# Salvar a tabela pivotada "analitic_SaldoPorCpf_group" na coluna B, linhas 2 a 11
analitic_SaldoPorCpf_group.to_excel(writer, sheet_name='Full_Bases', startrow=1, startcol=1)

# Salvar a tabela pivotada "analitic_telefones_bases_atraso" na coluna B, linhas 12 a 20
analitic_telefones_bases_atraso.to_excel(writer, sheet_name='Full_Bases', startrow=12, startcol=1)

# Fechar o escritor do Excel
writer.save()
